In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from sacrebleu import corpus_bleu

In [3]:
data = pd.read_csv('en-hi.csv')

In [4]:
data.shape

(10125706, 2)

In [5]:
data_1k = data.loc[:999]

In [6]:
data_1k.head()

,English,Hindi
0,"However, Paes, who was partnering Australia's ...",आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...
1,"Whosoever desires the reward of the world, wit...",और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...
2,The value of insects in the biosphere is enorm...,"जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ..."
3,Mithali To Anchor Indian Team Against Australi...,आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को
4,After the assent of the Honble President on 8t...,"8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व..."


In [7]:
data_1k.shape

(1000, 2)

In [8]:
# Initialize the translation model and tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

In [9]:
# Create an empty list to store translations
translations = []

# Iterate through each row in the DataFrame
for index, row in data_1k.iterrows():
    # Get the English statement from the DataFrame column
    english_statement = row['English']  
    # Convert the English statement to tensors
    model_inputs = tokenizer(english_statement, return_tensors="pt", padding=True, truncation=True)
    # Translate from English to Hindi
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
    )

    # Decode the generated tokens and append to the translations list
    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    translations.append(translation)

In [10]:
# Add the translations to a new column in the DataFrame
data_1k['Hindi_Translation'] = translations

In [11]:
data_1k.shape

(1000, 3)

In [12]:
data_1k.head()

,English,Hindi,Hindi_Translation
0,"However, Paes, who was partnering Australia's ...",आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाल...,"हालांकि पेस, जो ऑस्ट्रेलिया के पॉल हनले के साथ..."
1,"Whosoever desires the reward of the world, wit...",और जो शख्स (अपने आमाल का) बदला दुनिया ही में च...,"जो कोई दुनिया का बदला चाहता है, तो अल्लाह के प..."
2,The value of insects in the biosphere is enorm...,"जैव-मंडल में कीड़ों का मूल्य बहुत है, क्योंकि ...",जीवमंडल में कीटों का मूल्य बहुत बड़ा है क्योंक...
3,Mithali To Anchor Indian Team Against Australi...,आस्ट्रेलिया के खिलाफ वनडे टीम की कमान मिताली को,ओडीआई में मिताली ऑस्ट्रेलिया के खिलाफ भारतीय ट...
4,After the assent of the Honble President on 8t...,"8 सितम्‍बर, 2016 को माननीय राष्‍ट्रपति की स्‍व...","8 सितंबर, 2016 को माननीय राष्ट्रपति की अनुमति ..."


In [13]:
references = data_1k['Hindi'].tolist()
hypotheses = data_1k['Hindi_Translation'].tolist()

# Compute BLEU score
bleu = corpus_bleu(hypotheses, [references])
print(f'BLEU Score: {bleu.score}')

BLEU Score: 16.628108133043217


In [14]:
data_1k_2k = data.iloc[1000:2000]

In [15]:
data_1k_2k.shape

(1000, 2)

In [16]:
data_1k_2k.head()

,English,Hindi
1000,"US Secretary of State, Mike Pompeo","माइक पोंपियो , अमेरिकी विदेश मंत्री&nbsp"
1001,Sridevi was with her husband Boney Kapoor and ...,श्रीदेवी की मौत के वक्त उनके पति बोनी कपूर और ...
1002,Referring to an official communication receive...,"केंद्रीय राज्य मंत्री, पर्यटन और संस्कृति प्रह..."
1003,The former CM said:,जिनमें पूर्व मुख्यमंत्री चौ.
1004,Karimpur's seat fell vacant after sitting TMC ...,महुआ मोइत्रा के कृष्णानगर लोकसभा सीट से चुनाव ...


In [17]:
# Batch processing
batch_size = 32  # Experiment with different batch sizes
translations_1k_2k = []

for i in range(0, len(data_1k), batch_size):
    batch = data_1k_2k['English'][i:i + batch_size].tolist()

    model_inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)

    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"]
    )

    batch_translations = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    translations_1k_2k.extend(batch_translations)

In [18]:
data_1k_2k['Hindi_Translation'] = translations_1k_2k

In [19]:
data_1k_2k.head()

,English,Hindi,Hindi_Translation
1000,"US Secretary of State, Mike Pompeo","माइक पोंपियो , अमेरिकी विदेश मंत्री&nbsp",अमरीकी विदेश मंत्री माइक पॉम्पियो
1001,Sridevi was with her husband Boney Kapoor and ...,श्रीदेवी की मौत के वक्त उनके पति बोनी कपूर और ...,श्रीदेवी मृत्यु के समय अपने पति बोनी कपूर और ब...
1002,Referring to an official communication receive...,"केंद्रीय राज्य मंत्री, पर्यटन और संस्कृति प्रह...","केंद्रीय राज्य, पर्यटन और संस्कृति मंत्री, प्र..."
1003,The former CM said:,जिनमें पूर्व मुख्यमंत्री चौ.,पूर्व प्रधानमंत्री ने कहा थाः
1004,Karimpur's seat fell vacant after sitting TMC ...,महुआ मोइत्रा के कृष्णानगर लोकसभा सीट से चुनाव ...,करीमपुर का स्थान टीएमसी एमएलए माहू मोइत्रा के ...


In [20]:
references1 = data_1k_2k['Hindi'].tolist()
hypotheses1 = data_1k_2k['Hindi_Translation'].tolist()

# Compute BLEU score
bleu1 = corpus_bleu(hypotheses1, [references1])
print(f'BLEU Score: {bleu1.score}')

BLEU Score: 18.64978249391304


In [21]:
data_1k_2k.iloc[4].tolist()

["Karimpur's seat fell vacant after sitting TMC MLA Mahua Moitra won the parliamentary polls from Krishnnanagar Lok Sabha seat.",
 'महुआ मोइत्रा के कृष्णानगर लोकसभा सीट से चुनाव जीतने के बाद करीमपुर सीट खाली हो गई थी।',
 'करीमपुर का स्थान टीएमसी एमएलए माहू मोइत्रा के बैठने के बाद रिक्त हो गया और कृष्णानगर लोक सभा के स्थान से संसदीय चुनाव जीत गया।']

In [22]:
data_1k.iloc[0].tolist()

["However, Paes, who was partnering Australia's Paul Hanley, could only go as far as the quarterfinals where they lost to Bhupathi and Knowles",
 'आस्ट्रेलिया के पाल हेनली के साथ जोड़ी बनाने वाले पेस मियामी में क्वार्टरफाइनल तक ही पहुंच सके क्योंकि इस दौर में उन्हें भूपति और नोल्स ने हराया था।',
 'हालांकि पेस, जो ऑस्ट्रेलिया के पॉल हनले के साथ भागीदारी कर रहे थे, केवल क्वार्टफाइनल तक पहुँच सकते थे जहां वे भुपाती और नोल्स से हार गए']